# I. Introduction
In this note we go through the concept of Kalman filter, the notebook is heavily inspired by lessons from [Udacity course](https://www.udacity.com/drive).

The notebook is organized as following

* Re-call on Gaussian distribution
* Kalman filter in 1D space
* 

# II. Gaussian distribution
The Gaussian distribution has following form for 1D variable

$$
\mathcal{N}(x\ |\ \mu,\sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp\left\{-\frac{(x-\mu)^2}{2\sigma^2}\right\}
$$

where $\mu$ is the mean and $\sigma^2$ is the variance. 

For $D-$dimension, it takes the form

$$
\mathcal{N}(x\ |\ \mu,\Sigma^2) = \frac{1}{(2\pi)^{D/2}|\Sigma|^{1/2}}\exp\left\{-\frac{1}{2}(x-\mu)^T\Sigma^{-1}(x-\mu)\right\}
$$

### Conditional/Marginal Gaussian
Consider $\mathcal{N}(x\ |\ \mu,\Sigma)$ with $\Lambda=\Sigma^{-1}$ and

$$
\begin{split}
&x =\left(\begin{array}{c}x_a\\ x_b\end{array}\right),\ &\mu=\left(\begin{array}{c}\mu_a\\ \mu_b\end{array}\right)\\
&\Sigma = \left(\begin{array}{cc}\Sigma_{aa} & \Sigma_{ab}\\ \Sigma_{ba} & \Sigma_{bb}\end{array}\right),\ 
&\Lambda= \left(\begin{array}{cc}\Lambda_{aa} & \Lambda_{ab}\\ \Lambda_{ba} & \Lambda_{bb}\end{array}\right)\end{split}
$$

Then we have
* Conditional distribution
$$
\begin{split}
p(x_a|x_b) &= \mathcal{N}\left(x\ |\ \mu_{a|b},\Lambda_{aa}^{-1}\right)\\
\mu_{a|b} &= \mu_a-\Lambda_{aa}^{-1}\Lambda_{ab}(x_b - \mu_b)
\end{split}
$$
* Marginal distribution
$$
p(x_a) = \mathcal{N}\left(x_a\ |\ \mu_a, \Sigma_{aa}\right)
$$

### Bayes' theorem for Gaussian variables
We recall Bayes' theorem
$$
\begin{split}
\mathbb{P}(X,Y) = \mathbb{P}(X\ |\ Y)\mathbb{P}(Y)=\mathbb{P}(Y\ |\ X)\mathbb{P}(X)\\
\end{split}
$$

Given the prior and conditional distribution (linear Gaussian model)
$$
\begin{split}
p(x) &= \mathcal{N}\left(x\ |\ \mu,\Lambda^{-1}\right)\\
p(y\ |\ x) &= \mathcal{N}\left(y\ |\ Ax + b, L^{-1}\right)
\end{split}
$$
One can apply Bayes' theorem to find the marginal distribution $p(y)$ and the conditional distribution (also called posteriori) distribution $p(x\ |\ y)$.

We have (proof can be found in chapter 2 in Bishop's book)
$$
\begin{split}
p(y)   &= \mathcal{N}\left(y\ |\ A\mu+b,L^{-1} + A\Lambda^{-1}A^T\right)\\
p(x|y) &= \mathcal{N}\left(x\ |\ \Sigma\left\{A^TL(y-b) + \Lambda\mu\right\}, \Sigma\right)
\end{split}
$$
where
$$
\Sigma=\left(\Lambda + A^TLA\right)^{-1}
$$

# III. Kalman filter
The [Kalman filter](https://en.wikipedia.org/wiki/Kalman_filter) is an algorithm to estimate current state by iterating on two main cycles: mesurement update/motion update (also called update step/prediction step)

* **Measurement update**: given a measurement, we update the distribution of the current state
* **Motion update**: given a physical update (e.g move forward 10m), we predict the distribution of the next state

Let's go through some example (taken for Udacity course), consider a robot and a line of 5 squares (each is either green or red) as the following image

![robot_loc](assets/kalman_locs.png)

The robot can be in any square with uniform probability 0.2, denote $X$ as our robot's the location

$$
P(X=i) = 0.2
$$

We know that our robot can sense its location's color, denote $Y$ as the sensed color, we have $Y\in\left\{\text{red, green}\right\}$

### Measurement update
Now, if the robot sensed the color red $Y=\text{red}$, what is the distribution of robot's location i.e 
$$
P(X=i\ |\ Y=\text{red}) = ?
$$
By applying Bayes' rule, the distribution of robot's location becomes
$$
P(X=i\ |\ Y=\text{red}) = \left\{ \begin{array}{l}
                                    0.5 \text{ for } i=2,3 \\
                                    0 \text{ otherwise}
                                  \end{array}\right.
$$
So with an additional info (color sensing), we know more precise about our robot's location. This is the **Measurement update** step.

### Motion update
Now, let's our robot to move one step to the right, given previous step's distribution we know that
$$
P(X=i) = \left\{ \begin{array}{l}
                  0.5 \text{ for } i=3,4 \\
                  0 \text{ otherwise}
                  \end{array}\right.
$$
If we sense the color **red** again, applying another **Measurement update** then we know for sure that our robot's current location must be 3.

So we go through an example where robot's location is discrete and measurement/motion update is perfect (no noise). The Kalman filter is much more powerful and it can handle 

* state is continuous (1D or multi-dimensional)
* measurement can have Gaussian noise
* motion update can have Gaussian noise

Let's dive into 1D-case
### 1D Kalman filter
Now we consider a car that drives on a straight line with constant speed $v$ and we denote 

* $x_t$ is the real position of the car at time $t$ (which we don't know exactly) and 
* $y_t$ is position obtained via GPS (which might have small errror/noise)

We also assume the initial location $x_0\sim\mathcal{N}(\mu_0, \sigma_0^2)$, and all noise/error is of Gaussian distribution i.e
$$
\begin{split}
y_t &= x_t + \epsilon,&\quad \epsilon\sim\mathcal{N}(0, \theta^2)\\
x_t &= x_{t-1} + v + \xi,&\quad \xi \sim\mathcal{N}(0, \sigma^2)
\end{split}
$$

The goal is to estimate $x_t$ given observed $y_t$. Since the noise/error are random, we can only estimate the distribution of $x_t$ i.e
$$
\mathbb{P}(x_t\ |\ y_t)
$$
This is done in two steps, suppose we have from previous step $x_{t-1}\sim\mathcal{N}(\mu_{t-1}, \sigma_{t-1}^2)$
* Motion update
$$
x_t = x_{t-1} + v + \xi \sim \mathcal{N}(x_t \ |\ \mu_{t-1} + v, \sigma_{t-1}^2 + \sigma^2)
$$
* Measurement update, we know that
$$
p(y_t\ |\ x_t) \sim \mathcal{N}(y_t\ |\ x_t, \theta^2)
$$

So we have
$$
\begin{split}
p(x_t) & \sim \mathcal{N}(x_t \ |\ \mu_{t-1} + v, \sigma_{t-1}^2 + \sigma^2)\\
p(y_t\ |\ x_t) & \sim \mathcal{N}(y_t \ |\ x_t, \theta^2)
\end{split}
$$
Applying Bayes' theorem for Gaussian, we obtain
$$
\begin{split}
p(y_t) &\sim \\
p(x_t\ |\ y_t) &\sim 
\end{split}
$$